[Rendered Notebook](http://nbviewer.jupyter.org/github/andersonfrailey/Notebook-Uploads/blob/master/newcpswts.ipynb)

In [1]:
import pandas as pd
import numpy as np
import taxcalc as tc
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, NumeralTickFormatter
from bokeh.layouts import layout, row, gridplot, column
from notebookfunctions import distribution, index_list, percentile
output_notebook()

Loading BokehJS ...

In [2]:
years = [i for i in range(2014, 2028)]

In [3]:
cps = pd.read_csv('/Users/andersonfrailey/Tax-Calculator/taxcalc/cps.csv.gz', compression='gzip')
new_wts = pd.read_csv('cps_stage2/cps_weights.csv.gz', compression='gzip')

In [4]:
new_wts.head()

,WT2014,WT2015,WT2016,WT2017,WT2018,WT2019,WT2020,WT2021,WT2022,WT2023,WT2024,WT2025,WT2026,WT2027
0,23225,23519,23890,24301,24681,25014,25309,25608,25895,26185,26477,26774,27075,27377
1,19649,19898,20211,20559,20881,21163,21412,21665,21908,22153,22400,22651,22906,23161
2,27419,28361,29303,30272,31287,32350,33466,34564,35707,36838,37939,39085,40169,41182
3,17299,26277,17794,18101,18383,18632,18851,19074,19288,19504,19721,19942,20167,20391
4,29079,30078,31077,32104,33181,34308,35491,36657,37869,39068,40236,41451,42600,43675


In [5]:
base = tc.Calculator(records=tc.Records.cps_constructor(),
                     policy=tc.Policy())
new = tc.Calculator(records=tc.Records(data=cps, weights=new_wts, start_year=2014),
                    policy=tc.Policy())

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [6]:
# collect data for each of the years
base_data = {}
new_data = {}
var_list = ['s006', 'iitax', 'combined', 'payrolltax', 'c00100',
            'expanded_income']
for year in years:
    base.advance_to_year(year)
    base.calc_all()
    new.advance_to_year(year)
    new.calc_all()
    base_data[year] = base.dataframe(var_list)
    new_data[year] = new.dataframe(var_list)

In [7]:
# tax liability by year in 2014
data = {'year': [], 'combined_base': [], 'combined_new': [], 'combined_change': [],
        'combined_pct_change': [], 'iitax_pct_change': [], 'payroll_pct_change': [],
        'iitax_base': [], 'iitax_new': [], 'iitax_change': [],
        'payroll_base': [], 'payroll_new': [], 'payroll_change': []}
for year in years:
    data['year'].append(year)
    combined_new = (new_data[year]['combined'] * new_data[year]['s006']).sum()
    combined_base = (base_data[year]['combined'] * base_data[year]['s006']).sum()
    combined_change = combined_new - combined_base
    combined_pct_change = (combined_change / combined_base) * 100
    
    iitax_new = (new_data[year]['iitax'] * new_data[year]['s006']).sum()
    iitax_base = (base_data[year]['iitax'] * base_data[year]['s006']).sum()
    iitax_change = iitax_new - iitax_base
    iitax_pct_change = (iitax_change / iitax_base) * 100

    payroll_new = (new_data[year]['payrolltax'] * new_data[year]['s006']).sum()
    payroll_base = (base_data[year]['payrolltax'] * base_data[year]['s006']).sum()
    payroll_change = payroll_new - payroll_base
    payroll_pct_change = (payroll_change / payroll_base) * 100

    data['combined_base'].append(combined_base)
    data['combined_new'].append(combined_new)
    data['combined_change'].append(combined_change)
    data['combined_pct_change'].append(combined_pct_change)
    data['iitax_base'].append(iitax_base)
    data['iitax_new'].append(iitax_new)
    data['iitax_change'].append(iitax_change)
    data['iitax_pct_change'].append(iitax_pct_change)
    data['payroll_base'].append(payroll_base)
    data['payroll_new'].append(payroll_new)
    data['payroll_change'].append(payroll_change)
    data['payroll_pct_change'].append(payroll_pct_change)
cds = ColumnDataSource(data)
data_df = pd.DataFrame(data, index=data['year'])

In [8]:
f = figure(title='Tax Liability by Year')
f.line(x='year', y='combined_base', line_width=2, color='blue', source=cds, legend='Combined-Base')
f.line(x='year', y='combined_new', line_width=2, color='lightblue', source=cds, legend='Combined-new')
f.line(x='year', y='iitax_base', line_width=2, color='green', source=cds, legend='IITax-Base')
f.line(x='year', y='iitax_new', line_width=2, color='lime', source=cds, legend='IITax-New')
f.line(x='year', y='payroll_base', line_width=2, color='red', source=cds, legend='Payroll-Base')
f.line(x='year', y='payroll_new', line_width=2, color='pink', source=cds, legend='Payroll-New')
f.legend.location='top_left'
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
f.xaxis[0].ticker.desired_num_ticks = 12
f.xaxis.minor_tick_line_color = None
show(f)

In [9]:
pd.options.display.float_format = '{:,.2f}'.format
print('Combined Tax Liability')
data_df[['combined_base', 'combined_new', 'combined_change', 'combined_pct_change']].transpose()

Combined Tax Liability


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
combined_base,"2,049,761,794,852.92","2,110,841,250,755.81","2,192,101,336,640.05","2,328,589,406,483.97","2,326,852,347,902.63","2,455,208,000,725.77","2,578,843,260,672.17","2,711,805,209,706.87","2,853,027,976,383.55","3,001,258,288,406.50","3,155,871,943,780.05","3,317,717,564,356.19","3,682,346,347,868.92","3,862,491,374,617.44"
combined_new,"1,941,750,040,546.99","2,271,690,936,870.95","2,347,085,334,260.75","2,479,418,762,478.26","2,457,484,512,690.08","2,581,454,789,579.83","2,700,165,928,804.79","2,827,084,110,131.07","2,962,020,856,424.67","3,103,050,844,271.33","3,247,337,684,766.05","3,390,302,243,030.46","3,756,037,098,960.04","3,918,888,283,719.68"
combined_change,"-108,011,754,305.92","160,849,686,115.13","154,983,997,620.70","150,829,355,994.29","130,632,164,787.45","126,246,788,854.06","121,322,668,132.62","115,278,900,424.20","108,992,880,041.12","101,792,555,864.83","91,465,740,986.00","72,584,678,674.27","73,690,751,091.12","56,396,909,102.24"
combined_pct_change,-5.27,7.62,7.07,6.48,5.61,5.14,4.70,4.25,3.82,3.39,2.90,2.19,2.00,1.46


In [10]:
print('Income Tax Liability')
data_df[['iitax_base', 'iitax_new', 'iitax_change', 'iitax_pct_change']].transpose()

Income Tax Liability


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
iitax_base,"1,052,768,062,368.58","1,079,321,792,276.94","1,118,073,598,335.02","1,194,682,943,874.67","1,134,225,986,055.48","1,207,623,350,219.63","1,278,600,912,769.22","1,355,456,344,576.62","1,435,537,108,564.83","1,518,783,771,055.15","1,605,135,131,486.43","1,695,298,234,281.98","1,984,937,041,617.15","2,086,678,633,847.68"
iitax_new,"1,008,720,583,848.63","1,202,226,585,586.57","1,239,149,637,816.91","1,313,705,269,307.49","1,236,986,326,416.77","1,311,141,901,323.80","1,383,626,797,718.44","1,461,563,882,221.55","1,543,359,589,580.43","1,628,136,753,597.27","1,713,727,151,569.36","1,795,697,763,985.66","2,098,164,719,867.73","2,195,369,456,455.77"
iitax_change,"-44,047,478,519.95","122,904,793,309.63","121,076,039,481.90","119,022,325,432.82","102,760,340,361.29","103,518,551,104.17","105,025,884,949.22","106,107,537,644.93","107,822,481,015.60","109,352,982,542.12","108,592,020,082.93","100,399,529,703.68","113,227,678,250.58","108,690,822,608.10"
iitax_pct_change,-4.18,11.39,10.83,9.96,9.06,8.57,8.21,7.83,7.51,7.20,6.77,5.92,5.70,5.21


In [11]:
print('Payroll Tax Liability')
data_df[['payroll_base', 'payroll_new', 'payroll_change', 'payroll_pct_change']].transpose()

Payroll Tax Liability


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
payroll_base,"996,993,732,484.34","1,031,519,458,478.87","1,074,027,738,305.03","1,133,906,462,609.30","1,192,626,361,847.16","1,247,584,650,506.14","1,300,242,347,902.95","1,356,348,865,130.26","1,417,490,867,818.72","1,482,474,517,351.36","1,550,736,812,293.63","1,622,419,330,074.21","1,697,409,306,251.77","1,775,812,740,769.76"
payroll_new,"933,029,456,698.36","1,069,464,351,284.38","1,107,935,696,443.84","1,165,713,493,170.77","1,220,498,186,273.32","1,270,312,888,256.03","1,316,539,131,086.35","1,365,520,227,909.53","1,418,661,266,844.24","1,474,914,090,674.07","1,533,610,533,196.69","1,594,604,479,044.80","1,657,872,379,092.31","1,723,518,827,263.91"
payroll_change,"-63,964,275,785.98","37,944,892,805.50","33,907,958,138.80","31,807,030,561.48","27,871,824,426.16","22,728,237,749.89","16,296,783,183.40","9,171,362,779.27","1,170,399,025.52","-7,560,426,677.29","-17,126,279,096.94","-27,814,851,029.41","-39,536,927,159.46","-52,293,913,505.86"
payroll_pct_change,-6.42,3.68,3.16,2.81,2.34,1.82,1.25,0.68,0.08,-0.51,-1.10,-1.71,-2.33,-2.94


In [12]:
def hist_plots(base_data, new_data, year):
    change_wts = new_data['s006'] - base_data['s006']
    pct_change_wts = (change_wts / base_data['s006']) * 100
    hist_diff, edges_diff = np.histogram(change_wts, density=True, bins=50)
    hist_pct, edges_pct = np.histogram(pct_change_wts, density=True, bins=50)
    
    # create plots
    f1 = figure(title=f'Change in Weights for {year}', width=350, height=350)  # difference
    f1.quad(top=hist_diff, bottom=0, left=edges_diff[:-1], right=edges_diff[1:],
            fill_color='cyan', line_color='black')
    f1.yaxis.axis_label = 'Frequency'
    f1.xaxis.axis_label = 'Change'

    f2 = figure(title=f'Pct Change in Weights for {year}', width=350, height=350)  # pct change
    f2.quad(top=hist_pct, bottom=0, left=edges_pct[:-1], right=edges_pct[1:],
            fill_color='cyan', line_color='black')
    f2.yaxis.axis_label = 'Frequency'
    f2.xaxis.axis_label = 'Pct Change'
    
    return f1, f2

In [13]:
# analyze change in each record's weight
plot_list_diff = []  # holds graphs showing difference in wt
plot_list_pct = []  # holds graphs showing histogram of pct change in wt
for year in years:
    base_data1 = base_data[year]
    new_data1 = new_data[year]

    hist_diff1, hist_pct1 = hist_plots(base_data1, new_data1, year)
    plot_list_diff.append(hist_diff1)
    plot_list_pct.append(hist_pct1)

In [14]:
show(column(plot_list_diff))

In [15]:
show(column(plot_list_pct))

### Add PUF Data for Comparison

In [16]:
puf_path = '/Users/andersonfrailey/Tax-Calculator/puf.csv'
calc_puf = tc.Calculator(records=tc.Records(data=puf_path), policy=tc.Policy())

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [17]:
combined = []
iitax = []
payroll = []
for year in years:
    calc_puf.advance_to_year(year)
    calc_puf.calc_all()
    combined.append(calc_puf.weighted_total('combined'))
    iitax.append(calc_puf.weighted_total('iitax'))
    payroll.append(calc_puf.weighted_total('payrolltax'))

In [18]:
cds_puf = ColumnDataSource({'combined': combined,
                            'iitax': iitax,
                            'payroll': payroll,
                            'year': years})

In [19]:
f.line(x='year', y='combined', line_width=2, color='navy', legend='Combined-PUF', source=cds_puf)
f.line(x='year', y='iitax', line_width=2, color='darkgreen', legend='IITax-PUF', source=cds_puf)
f.line(x='year', y='payroll', line_width=2, color='darkred', legend='Payroll-PUF', source=cds_puf)

bokeh.models.renderers.GlyphRenderer(
    id='65d05a0c-e308-4193-bc9e-2ed7f94528de',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='03e51dc6-eeb9-4121-ba19-cbfa672ab2a9',
        callback=None,
        data={'combined': [2320417525056.1162,
          2491788683919.7847,
          2572337580916.7598,
          2725401439685.4844,
          2669810142992.1367,
          2777417911535.0508,
          2883297994543.2847,
          2999523877167.4614,
          3128265046067.5044,
          3269568279451.8423,
          3417521728848.7637,
          3570882409629.5586,
          3966239038871.0654,
          4145087918985.0005],
         'iitax': [1363408032999.9075,
          1491075240575.9429,
          1534097676441.6125,
          1633153615224.1956,
          1528758034745.2034,
          1591688051898.6255,
          1655551746951.6526,
          1727100245168.1477,
          1807025001782.0508,
          1896234488555.5671,
          1989645356456.3804,
          2086434321976.321,
          2422729095825.9941,
          2539966119680.8071],
         'payroll': [957009492056.20837,
          1000713443343.8417,
          1038239904475.1475,
          1092247824461.2889,
          1141052108246.9338,
          1185729859636.4255,
          1227746247591.6326,
          1272423631999.3135,
          1321240044285.4541,
          1373333790896.2744,
          1427876372392.3843,
          1484448087653.2368,
          1543509943045.0713,
          1605121799304.1929],
         'year': [2014,
          2015,
          2016,
          2017,
          2018,
          2019,
          2020,
          2021,
          2022,
          2023,
          2024,
          2025,
          2026,
          2027]},
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        selected=bokeh.models.selections.Selection(
            id='fe8c5ea6-91eb-4c39-8b9b-14566e3d238a',
            indices=[],
            js_event_callbacks={},
            js_property_callbacks={},
            line_indices=[],
            multiline_indices={},
            name=None,
            subscribed_events=[],
            tags=[]),
        selection_policy=bokeh.models.selections.UnionRenderers(
            id='2b47bf89-a070-427a-8ded-76b7135bd789',
            js_event_callbacks={},
            js_property_callbacks={},
            name=None,
            subscribed_events=[],
            tags=[]),
        subscribed_events=[],
        tags=[]),
    glyph=bokeh.models.glyphs.Line(
        id='2c439fa9-cf67-493a-8259-0a2b2ad7a752',
        js_event_callbacks={},
        js_property_callbacks={},
        line_alpha=1.0,
        line_cap='butt',
        line_color='darkred',
        line_dash=[],
        line_dash_offset=0,
        line_join='miter',
        line_width=2,
        name=None,
        subscribed_events=[],
        tags=[],
        x={'field': 'year'},
        y={'field': 'payroll'}),
    hover_glyph=None,
    js_event_callbacks={},
    js_property_callbacks={},
    level='glyph',
    muted=False,
    muted_glyph=None,
    name=None,
    nonselection_glyph=bokeh.models.glyphs.Line(
        id='36b28ff9-b6de-408f-bb61-2c9f52f397e5',
        js_event_callbacks={},
        js_property_callbacks={},
        line_alpha=0.1,
        line_cap='butt',
        line_color='#1f77b4',
        line_dash=[],
        line_dash_offset=0,
        line_join='miter',
        line_width=2,
        name=None,
        subscribed_events=[],
        tags=[],
        x={'field': 'year'},
        y={'field': 'payroll'}),
    selection_glyph=None,
    subscribed_events=[],
    tags=[],
    view=bokeh.models.sources.CDSView(
        id='5ddd0409-2b28-45cd-9a51-391f91418ff3',
        filters=[],
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        source=bokeh.models.sources.ColumnDataSource(
            id='03e51dc6-eeb9-4121-ba19-cbfa672ab2a9',
            callback=None,
            data={'combined

In [20]:
show(f)

### Change in Weight by Tax Liability

In [21]:
def wt_plot(data, year, tax):
    f3 = figure(title=f'Average Change in Weight by Tax Liability - {year}')
    f3.circle(x='perc', y='change', size=5, source=data)
    f3.xaxis.axis_label = 'Tax Liability Percentile'
    f3.yaxis.axis_label = 'Average Weight Change'
    return(f3)

In [22]:
plots = []
data = []
for year in years:
    tax = 'combined'
    temp = pd.DataFrame({'wt_change': new_data[year]['s006'] - base_data[year]['s006'],
                         's006': base_data[year]['s006'],
                         tax: new_data[year][tax]})
    change = percentile(temp, 'wt_change', 100, 'combined')
    cds = ColumnDataSource({'perc': [i for i in range(1, 101)], 'change': change})
    plot = wt_plot(cds, year, 'Combined')
    plots.append(plot)

In [23]:
for p in plots:
    show(p)